# About Mini Project

Mini project for hearing impaired people, using tools, suggesting songs according to a certain genre and in sign language. Speech to text converter with multiple language support.

## Extra requirements
- pip install pydub simpleaudio speechrecognition pipwin pyaudio


In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
system_message = "You are a helpful assistant for hearing impaired people. "
system_message += "Your mission is convert text to speech and speech to text. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
songs_with_signs = {
    "electronic": ("God is a dj", "https://www.youtube.com/watch?v=bhSB8EEnCAM", "Faithless"), 
    "pop": ("Yitirmeden", "https://www.youtube.com/watch?v=aObdAXq1ZIo", "Pinhani"), 
    "rock": ("Bohemian Rhapsody", "https://www.youtube.com/watch?v=sjln9OMOw-0", "Queen")
}

def get_songs_with_sign_language(genre):
    print(f"Tool get_songs_with_sign_language called for {genre}")
    city = genre.lower()
    return songs_with_signs.get(genre, "Unknown")

In [ ]:
get_songs_with_sign_language("rock")

In [ ]:
song_function = {
    "name": "get_songs_with_sign_language",
    "description": "Get the corresponding song information for the specified given music genre. Call this whenever you need to know the songs with specific genre and in sign language, for example when a customer asks 'Suggest me sign language supported songs'",
    "parameters": {
        "type": "object",
        "properties": {
            "genre": {
                "type": "string",
                "description": "The music genre that the customer wants to listen-watch to",
            },
        },
        "required": ["genre"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": song_function}]

In [ ]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    genre = arguments.get('genre')
    song = get_songs_with_sign_language(genre)
    song_info = song[2] + ": "  + song[1]
    response = {
        "role": "tool",
        "content": json.dumps({"genre": genre,"song": song_info}),
        "tool_call_id": tool_call.id
    }
    return response, song[1]

In [ ]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    genre = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, genre = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]
    
    return history, genre

In [ ]:
import speech_recognition as sr
from pydub import AudioSegment
import simpleaudio as sa

def listener():
    recognizer = sr.Recognizer()
    
    with sr.Microphone() as source:
        print("Listening... Speak now!")
        recognizer.adjust_for_ambient_noise(source)  # Adjust for background noise
        audio = recognizer.listen(source)
    
    try:
        print("Processing speech...")
        text = recognizer.recognize_google(audio)  # Use Google Speech-to-Text
        print(f"You said: {text}")
        return text
    except sr.UnknownValueError:
        print("Sorry, I could not understand what you said.")
        return None
    except sr.RequestError:
        print("Could not request results, please check your internet connection.")
        return None

# Example usage:
text = listener()  # Listen for speech
if text:
    print(f"You just said: {text}") 

In [ ]:
import gradio as gr

convert = gr.State(False)
def toggle_convert(current_value):
    return not current_value

In [ ]:
import gradio as gr

with gr.Blocks() as ui:
    with gr.Tab("Chat") as chat_interface:
        with gr.Row():
            chatbot = gr.Chatbot(height=500, type="messages")
            video = gr.HTML(f"<a href=''> Example song will appear here </a>")
        with gr.Row():
            entry = gr.Textbox(label="Chat with our AI Assistant:")
        with gr.Row():
            clear = gr.Button("Clear")
    
        def do_entry(message, history):
            history += [{"role":"user", "content":message}]
            return "", history
    
        entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
            chat, inputs=chatbot, outputs=[chatbot, video]
        )
        clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)
    with gr.Tab("Speech to text converter") as speech_to_text:
        text_output = gr.Markdown("Press the button to start voice recognition")
        listen_button = gr.Button("Convert Voice to Text")
        language = gr.Dropdown(["English", "Turkish", "Greek", "Arabic"], label="Select output language", value="English")

        def update_text(language):
            """Calls the listener and updates the markdown output in specific language."""
            text = listener()  # Replace with real speech-to-text function
            system_prompt = f"You are a useful translator. Convert text to {language}. Do not add aditional data, only translate it."
            response = openai.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": text}
                ],
            )
            return f"**Converted Text:** {response.choices[0].message.content}"

        listen_button.click(update_text, inputs=[language], outputs=[text_output])

ui.launch(inbrowser=True, share=True)

